In [1]:
%load_ext twisted_ipython
%autoawait twisted

In [2]:
from twisted.internet import reactor

import attr
from txdbus import client

from pyxsession.dbus.marshmallow.schema import DBusSchema
from pyxsession.dbus.marshmallow.fields import (
    DBusField, Int32, UInt32, Str, List, Tuple
)
from pyxsession.dbus.signature import signature
      
@attr.s
class Notification:
    appname = attr.ib(default='')
    replaces = attr.ib(default=0)
    icon = attr.ib(default='')
    summary = attr.ib(default='')
    message = attr.ib(default='')
    actions = attr.ib(default=[])
    hints = attr.ib(default={})
    timeout = attr.ib(default=1000)
  
class NotificationSchema(DBusSchema):
    cls = Notification
  
    appname = Str()
    replaces = UInt32()
    icon = Str()
    summary = Str()
    message = Str()
    actions = DBusField('as')
    hints = DBusField('a{sv}')
    timeout = Int32()
    
schema = NotificationSchema()

In [3]:
notification = Notification(
    appname='jupyter',
    summary='an important message',
    message='my dog',
    timeout=5000,
    hints={'key': 'bro'}
)

In [4]:
dump = schema.dump(notification)

dump

['jupyter', 0, '', 'an important message', 'my dog', [], {'key': 'bro'}, 5000]

In [5]:
load = schema.load(dump)

load

Notification(appname='jupyter', replaces=0, icon='', summary='an important message', message='my dog', actions=[], hints={'key': 'bro'}, timeout=5000)

In [6]:
signature(schema)

'susssasa{sv}i'

In [7]:
conn = await client.connect(reactor, 'session')

notifier = await conn.getRemoteObject('org.freedesktop.Notifications', '/org/freedesktop/Notifications')

In [8]:
from txdbus.interface import DBusInterface, Method

iface = DBusInterface(
    'org.freedesktop.Notifications',
    Method('Notify', arguments=signature(schema), returns='u')
)

unreflected = await conn.getRemoteObject(
    'org.freedesktop.Notifications',
    '/org/freedesktop/Notifications',
    iface
)

await unreflected.callRemote('Notify', *schema.dump(Notification(appname='jupyter', summary='an important message', message='my dog', timeout=5000)))

17